In [0]:
import pyspark

In [0]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [0]:
sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [0]:
sentenceDataFrame.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard about ...|
 1|I wish Java could...|
 2|Logistic,regressi...|
+---+--------------------+

In [0]:
tokenizer=Tokenizer(inputCol='sentence',outputCol='words')

In [0]:
regex_tokenizer=RegexTokenizer(inputCol='sentence',outputCol='words',pattern='\\W')

In [0]:
count_tokens=udf(lambda words:len(words),IntegerType())

In [0]:
tokenized=tokenizer.transform(sentenceDataFrame)

In [0]:
tokenized.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard about ...|[hi, i, heard, ab...|
 1|I wish Java could...|[i, wish, java, c...|
 2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+

In [0]:
tokenized.withColumn("tokens",count_tokens(col("words"))).show(truncate=False)

+---+-----------------------------------+------------------------------------------+------+
id |sentence |words |tokens|
+---+-----------------------------------+------------------------------------------+------+
0 |Hi I heard about Spark |[hi, i, heard, about, spark] |5 |
1 |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7 |
2 |Logistic,regression,models,are,neat|[logistic,regression,models,are,neat] |1 |
+---+-----------------------------------+------------------------------------------+------+

In [0]:
regindexed=regex_tokenizer.transform(sentenceDataFrame)

In [0]:
regindexed.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hi I heard about ...|[hi, i, heard, ab...|
 1|I wish Java could...|[i, wish, java, c...|
 2|Logistic,regressi...|[logistic, regres...|
+---+--------------------+--------------------+

In [0]:
regindexed.withColumn("tokens",count_tokens(col("words"))).show(truncate=False)

+---+-----------------------------------+------------------------------------------+------+
id |sentence |words |tokens|
+---+-----------------------------------+------------------------------------------+------+
0 |Hi I heard about Spark |[hi, i, heard, about, spark] |5 |
1 |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7 |
2 |Logistic,regression,models,are,neat|[logistic, regression, models, are, neat] |5 |
+---+-----------------------------------+------------------------------------------+------+

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])


In [0]:
remover=StopWordsRemover(inputCol='raw',outputCol='filtered')

In [0]:
remover.transform(sentenceData).show(truncate=False)

+---+----------------------------+--------------------+
id |raw |filtered |
+---+----------------------------+--------------------+
0 |[I, saw, the, red, balloon] |[saw, red, balloon] |
1 |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+

In [0]:
from pyspark.ml.feature import NGram

In [0]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [0]:
ngram=NGram(n=2,inputCol="words",outputCol="ngrams",)

In [0]:
ngram.transform(wordDataFrame).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
id |words |ngrams |
+---+------------------------------------------+------------------------------------------------------------------+
0 |[Hi, I, heard, about, Spark] |[Hi I, I heard, heard about, about Spark] |
1 |[I, wish, Java, could, use, case, classes]|[I wish, wish Java, Java could, could use, use case, case classes]|
2 |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat] |
+---+------------------------------------------+------------------------------------------------------------------+

In [0]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [0]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish Java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [0]:
tokenizer=Tokenizer(inputCol="sentence",outputCol="words")

In [0]:
wordsData=tokenizer.transform(sentenceData)

In [0]:
wordsData.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
label|sentence |words |
+-----+-----------------------------------+------------------------------------------+
0.0 |Hi I heard about Spark |[hi, i, heard, about, spark] |
0.0 |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
1.0 |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

In [0]:
hashingTf=HashingTF(inputCol="words",outputCol="rawfeatures")

In [0]:
featurizedData=hashingTf.transform(wordsData)

In [0]:
idf=IDF(inputCol="rawfeatures",outputCol="features")

In [0]:
idfModel=idf.fit(featurizedData)

In [0]:
rescaledData=idfModel.transform(featurizedData)

In [0]:
rescaledData.select("label", "features").show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262144,[18700,19...|
 0.0|(262144,[19036,20...|
 1.0|(262144,[46243,58...|
+-----+--------------------+

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [0]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [0]:
model=cv.fit(df)

In [0]:
result=model.transform(df)

In [0]:
result.show(truncate=False)


+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+